In [1]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as np

In [2]:
urls = {'serie_a' : 'https://www.oddsportal.com/soccer/brazil/serie-a/',
'serie_b' : 'https://www.oddsportal.com/soccer/brazil/serie-b/',
'copa_brasil' : 'https://www.oddsportal.com/soccer/brazil/copa-do-brasil/',
'libertadores' : 'https://www.oddsportal.com/soccer/south-america/copa-libertadores/',
'sulamericana' :  'https://www.oddsportal.com/soccer/south-america/copa-sudamericana/'}

 

In [3]:
firefox = webdriver.Firefox(executable_path=r"./geckodriver.exe")

C:\Users\VINICI~1\AppData\Local\Temp/ipykernel_26428/170262964.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  firefox = webdriver.Firefox(executable_path=r"./geckodriver.exe")


In [4]:
soups = {}
for var, url in zip(urls.keys(), urls.values()):
    firefox.get(url)
    time.sleep(3)
    soups[var] = BeautifulSoup(firefox.page_source, 'html.parser')

firefox.close()

In [6]:
data = []
for var, soup in zip(soups.keys(), soups.values()):


    rows = soup.find('table', {'id': 'tournamentTable',
                    'class': 'table-main'}).tbody.find_all('tr')


    # data = []
    for row in rows:
        r = []
        if not row.find_all('td', class_='odds-nowrp') == []:
            try:
                game = row.find(
                    'td', class_='name table-participant').text.replace('\xa0', '').split(' - ')
            except:
                next
            if not game == []:
                r.append(var)
                r.append(game[0])
                r.append(game[1])
                for d in row.find_all('td', class_='odds-nowrp'):
                    r.append(float(d.text))

            data.append(r)

    # odds.append(data)

In [7]:
df=pd.DataFrame(data, columns=['competition','home', 'away', 'odd_h', 'odd_d', 'odd_a'])

In [8]:
odds_sum = (1/df[['odd_h', 'odd_d', 'odd_a']
                 ].values).sum(axis=1).reshape([-1, 1])

probs = (np.round(100*(1/df[['odd_h', 'odd_d', 'odd_a']].values)/odds_sum,2))


df[['p_h', 'p_d', 'p_a']] = probs

df

,competition,home,away,odd_h,odd_d,odd_a,p_h,p_d,p_a
0,serie_a,Coritiba,Santos,2.45,3.11,3.11,38.83,30.59,30.59
1,serie_a,Goias,Avai,2.01,3.26,3.84,46.73,28.81,24.46
2,serie_a,Corinthians,Palmeiras,3.24,3.11,2.36,29.29,30.51,40.21
3,serie_a,Cuiaba,Juventude,2.20,3.03,3.58,42.72,31.02,26.26
4,serie_a,Botafogo RJ,Atletico GO,2.13,3.23,3.48,44.02,29.03,26.95
5,serie_a,Coritiba,Atletico-MG,4.00,3.36,1.93,23.46,27.93,48.62
6,serie_a,Ceara,Fortaleza,2.40,3.09,3.10,39.20,30.45,30.35
7,serie_a,Flamengo RJ,Athletico-PR,1.44,4.40,6.91,65.12,21.31,13.57
8,serie_a,Sao Paulo,Bragantino,2.06,3.33,3.61,45.68,28.26,26.07
9,serie_a,America MG,Santos,2.29,3.13,3.23,40.97,29.98,29.05
